In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1])

In [3]:
def true_1D_1(x): #True function for 1D_1 dy2/dx2 + dy/dx - 6y = 0; BC1: y(0)=2; BC2: dy/dx at (x=0) = -1;
    y = np.exp(2*x) + np.exp(-3*x)
    return y
    

In [4]:
loss_thresh = 0.005

x = np.linspace(0,2,100).reshape(-1,1)

bc1_x = x[0].reshape(-1,1)
bc1_y = true_1D_1(x[0]).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)
    

bc2_x = x[0].reshape(-1,1)
x_bc2_train = torch.from_numpy(bc2_x).float().to(device)
bc2_val = torch.tensor(-1.0,device=device)
bc2_val = bc2_val.view(1,1)

x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)
y_true = true_1D_1(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

In [5]:
def colloc_pts(N_f,seed):
    #Collocation Points
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,y)
    x01 = np.array([[0.0, 1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    
    x_coll_train = lb + (ub-lb)*sampling(N_f)
    x_coll_train = np.vstack((x_coll_train, bc1_x.reshape(-1,1))) # append training points to collocation points 

    return x_coll_train

In [6]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 

    
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_BC2(self,x_bc2,bc2_val):
        g = x_bc2.clone()             
        g.requires_grad = True
        y = self.forward(g)    
            
        y_x = autograd.grad(y,g,torch.ones([x_bc2.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        dy_dx = y_x[:,[0]]
        
        bc2 = dy_dx
        
        loss_bc2= self.loss_function(bc2,bc2_val)

        return loss_bc2
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        y_xx = autograd.grad(y_x,g,torch.ones(x_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        dy2_d2x = y_xx[:,[0]]
        
        f = dy2_d2x + dy_dx - 6*y
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_bc2,bc2_val,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_bc2 = self.loss_BC2(x_bc2,bc2_val)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + loss_bc2 + loss_f
        
        return loss_val
          
    'test neural network'
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re

In [7]:
def train_step(x_coll,f_hat):
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [8]:
def data_update(loss_np):
    train_loss.append(loss_np)

    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [9]:
def train_model(max_iter,rep):
    print(rep) 
    torch.manual_seed(rep*123)
    start_time = time.time()
    thresh_flag = 0

    for i in range(max_iter):
        x_coll = torch.from_numpy(colloc_pts(N_f,i*11)).float().to(device)
        f_hat = torch.zeros(x_coll.shape[0],1).to(device)
        train_step(x_coll,f_hat)
        
        loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [10]:
for tune_reps in range(5):
    max_reps = 10
    max_iter = 100
    label = "1D_SODE_Stan_tune"+str(tune_reps)

    N_f = 1000

    train_loss_full = []
    test_mse_full = []
    test_re_full = []
    
    elapsed_time= np.zeros((max_reps,1))

    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))

    for reps in range(max_reps):

        train_loss = []
        test_mse_loss = []
        test_re_loss =[]
    

        'Generate Training data'
        torch.manual_seed(reps*36)
         #Total number of collocation points 


        layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
        PINN = Sequentialmodel(layers)
        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())

        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lr_tune[tune_reps], 
                                  max_iter = 10, 
                                  max_eval = 15, 
                                  tolerance_grad = 1e-5, 
                                  tolerance_change = 1e-5, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')


        train_model(max_iter,reps)


        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)

        print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full, "test_re_loss": test_re_full, "Time": elapsed_time, "label": label}
    savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 4.355709 Test MSE 386.09895378475534 Test RE 1.0016405428896509
1 Train Loss 3.9303586 Test MSE 387.8154916031521 Test RE 1.0038646451497013
2 Train Loss 3.5400162 Test MSE 386.1633448110432 Test RE 1.0017240628953885
3 Train Loss 2.4067354 Test MSE 384.52696115544995 Test RE 0.99959938

0 Train Loss 4.3353167 Test MSE 386.08777322089713 Test RE 1.0016260401475199
1 Train Loss 4.1922407 Test MSE 388.61863269884225 Test RE 1.0049035772961783
2 Train Loss 3.4473376 Test MSE 384.9500142121163 Test RE 1.0001491093233785
3 Train Loss 2.388115 Test MSE 384.0414863042791 Test RE 0.9989681765836386
4 Train Loss 2.384179 Test MSE 383.90834819982115 Test RE 0.9987950022480022
5 Train Loss 2.3827786 Test MSE 383.7559288641121 Test RE 0.9985967117124851
6 Train Loss 2.381363 Test MSE 383.64846297685176 Test RE 0.9984568798718934
7 Train Loss 2.3813548 Test MSE 383.6508508574167 Test RE 0.998459987133226
8 Train Loss 2.381347 Test MSE 383.6542004106569 Test RE 0.998464345767617
9 Train Loss 2.3813448 Test MSE 383.65567889271335 Test RE 0.9984662696488041
10 Train Loss 2.3813436 Test MSE 383.65619216352496 Test RE 0.9984669375438067
11 Train Loss 2.3813393 Test MSE 383.65934610681603 Test RE 0.9984710416110566
12 Train Loss 2.381337 Test MSE 383.6603221222942 Test RE 0.9984723116473

0 Train Loss 3.7553072 Test MSE 386.39154580664496 Test RE 1.002020000702379
1 Train Loss 2.4646204 Test MSE 384.024940555657 Test RE 0.9989466569627751
2 Train Loss 2.3849897 Test MSE 383.8011146967154 Test RE 0.9986555005054246
3 Train Loss 2.3827763 Test MSE 383.8912011426074 Test RE 0.9987726966816243
4 Train Loss 2.3808906 Test MSE 383.70432121857857 Test RE 0.9985295636191419
5 Train Loss 2.3808742 Test MSE 383.6972995145104 Test RE 0.9985204271424483
6 Train Loss 2.3808665 Test MSE 383.6935617459035 Test RE 0.9985155636114827
7 Train Loss 2.3808615 Test MSE 383.6904561137201 Test RE 0.9985115225893569
8 Train Loss 2.380859 Test MSE 383.6886087920621 Test RE 0.9985091188624046
9 Train Loss 2.3808565 Test MSE 383.68718167786244 Test RE 0.9985072619036176
10 Train Loss 2.3808532 Test MSE 383.68390938498067 Test RE 0.9985030039881488
11 Train Loss 2.3808532 Test MSE 383.68390938498067 Test RE 0.9985030039881488
12 Train Loss 2.3808532 Test MSE 383.68390938498067 Test RE 0.9985030039

0 Train Loss 4.330962 Test MSE 386.4566759157487 Test RE 1.0021044473249634
1 Train Loss 4.2706785 Test MSE 387.8917806166834 Test RE 1.0039633777677035
2 Train Loss 3.3814733 Test MSE 385.5573450892829 Test RE 1.0009377598259492
3 Train Loss 2.4003317 Test MSE 383.07680868203386 Test RE 0.9977127286545912
4 Train Loss 2.3873093 Test MSE 383.3969581206095 Test RE 0.9981295516699871
5 Train Loss 2.380776 Test MSE 383.6450369098561 Test RE 0.998452421639775
6 Train Loss 2.380664 Test MSE 383.670177136553 Test RE 0.9984851353550622
7 Train Loss 2.38066 Test MSE 383.6730364717556 Test RE 0.9984888559965762
8 Train Loss 2.3806567 Test MSE 383.6751222851449 Test RE 0.9984915701025939
9 Train Loss 2.3806567 Test MSE 383.6751222851449 Test RE 0.9984915701025939
10 Train Loss 2.3806567 Test MSE 383.6751222851449 Test RE 0.9984915701025939
11 Train Loss 2.3806567 Test MSE 383.6751222851449 Test RE 0.9984915701025939
12 Train Loss 2.3806567 Test MSE 383.6751222851449 Test RE 0.9984915701025939
13

0 Train Loss 4.3028893 Test MSE 386.78953825595966 Test RE 1.0025359200581314
1 Train Loss 3.4507682 Test MSE 385.2419984069904 Test RE 1.0005283434667311
2 Train Loss 2.3881316 Test MSE 384.06515031306554 Test RE 0.9989989535028048
3 Train Loss 2.3821802 Test MSE 383.94957944484696 Test RE 0.9988486354341648
4 Train Loss 2.3811386 Test MSE 383.80928404606163 Test RE 0.9986661288255815
5 Train Loss 2.381098 Test MSE 383.8030812464705 Test RE 0.9986580589961885
6 Train Loss 2.3810952 Test MSE 383.80240603478524 Test RE 0.9986571805432519
7 Train Loss 2.3810952 Test MSE 383.80240603478524 Test RE 0.9986571805432519
8 Train Loss 2.3810952 Test MSE 383.80240603478524 Test RE 0.9986571805432519
9 Train Loss 2.3810952 Test MSE 383.80240603478524 Test RE 0.9986571805432519
10 Train Loss 2.3810952 Test MSE 383.80240603478524 Test RE 0.9986571805432519
11 Train Loss 2.3810952 Test MSE 383.80240603478524 Test RE 0.9986571805432519
12 Train Loss 2.3810952 Test MSE 383.80240603478524 Test RE 0.998

...

26 Train Loss 2.381109 Test MSE 383.80921145881985 Test RE 0.9986660343901076
27 Train Loss 2.381109 Test MSE 383.80921145881985 Test RE 0.9986660343901076
28 Train Loss 2.381109 Test MSE 383.80921145881985 Test RE 0.9986660343901076
29 Train Loss 2.381109 Test MSE 383.80921145881985 Test RE 0.9986660343901076
30 Train Loss 2.381109 Test MSE 383.80921145881985 Test RE 0.9986660343901076
31 Train Loss 2.381109 Test MSE 383.80921145881985 Test RE 0.9986660343901076
32 Train Loss 2.381109 Test MSE 383.80921145881985 Test RE 0.9986660343901076
33 Train Loss 2.381109 Test MSE 383.80921145881985 Test RE 0.9986660343901076
34 Train Loss 2.381109 Test MSE 383.80921145881985 Test RE 0.9986660343901076
35 Train Loss 2.381109 Test MSE 383.80921145881985 Test RE 0.9986660343901076
36 Train Loss 2.381109 Test MSE 383.80921145881985 Test RE 0.9986660343901076
37 Train Loss 2.381109 Test MSE 383.80921145881985 Test RE 0.9986660343901076
38 Train Loss 2.381109 Test MSE 383.80921145881985 Test RE 0.998

26 Train Loss nan Test MSE nan Test RE nan
27 Train Loss nan Test MSE nan Test RE nan
28 Train Loss nan Test MSE nan Test RE nan
29 Train Loss nan Test MSE nan Test RE nan
30 Train Loss nan Test MSE nan Test RE nan
31 Train Loss nan Test MSE nan Test RE nan
32 Train Loss nan Test MSE nan Test RE nan
33 Train Loss nan Test MSE nan Test RE nan
34 Train Loss nan Test MSE nan Test RE nan
35 Train Loss nan Test MSE nan Test RE nan
36 Train Loss nan Test MSE nan Test RE nan
37 Train Loss nan Test MSE nan Test RE nan
38 Train Loss nan Test MSE nan Test RE nan
39 Train Loss nan Test MSE nan Test RE nan
40 Train Loss nan Test MSE nan Test RE nan
41 Train Loss nan Test MSE nan Test RE nan
42 Train Loss nan Test MSE nan Test RE nan
43 Train Loss nan Test MSE nan Test RE nan
44 Train Loss nan Test MSE nan Test RE nan
45 Train Loss nan Test MSE nan Test RE nan
46 Train Loss nan Test MSE nan Test RE nan
47 Train Loss nan Test MSE nan Test RE nan
48 Train Loss nan Test MSE nan Test RE nan
49 Train Lo

55 Train Loss 2.380128 Test MSE 383.59712564288196 Test RE 0.9983900741446269
56 Train Loss 2.380128 Test MSE 383.59712564288196 Test RE 0.9983900741446269
57 Train Loss 2.380128 Test MSE 383.59712564288196 Test RE 0.9983900741446269
58 Train Loss 2.380128 Test MSE 383.59712564288196 Test RE 0.9983900741446269
59 Train Loss 2.380128 Test MSE 383.59712564288196 Test RE 0.9983900741446269
60 Train Loss 2.380128 Test MSE 383.59712564288196 Test RE 0.9983900741446269
61 Train Loss 2.380128 Test MSE 383.59712564288196 Test RE 0.9983900741446269
62 Train Loss 2.380128 Test MSE 383.59712564288196 Test RE 0.9983900741446269
63 Train Loss 2.380128 Test MSE 383.59712564288196 Test RE 0.9983900741446269
64 Train Loss 2.380128 Test MSE 383.59712564288196 Test RE 0.9983900741446269
65 Train Loss 2.380128 Test MSE 383.59712564288196 Test RE 0.9983900741446269
66 Train Loss 2.380128 Test MSE 383.59712564288196 Test RE 0.9983900741446269
67 Train Loss 2.380128 Test MSE 383.59712564288196 Test RE 0.998

54 Train Loss 2.3811495 Test MSE 383.83359083344834 Test RE 0.998697751277094
55 Train Loss 2.3811495 Test MSE 383.83359083344834 Test RE 0.998697751277094
56 Train Loss 2.3811495 Test MSE 383.83359083344834 Test RE 0.998697751277094
57 Train Loss 2.3811495 Test MSE 383.83359083344834 Test RE 0.998697751277094
58 Train Loss 2.3811495 Test MSE 383.83359083344834 Test RE 0.998697751277094
59 Train Loss 2.3811495 Test MSE 383.83359083344834 Test RE 0.998697751277094
60 Train Loss 2.3811495 Test MSE 383.83359083344834 Test RE 0.998697751277094
61 Train Loss 2.3811495 Test MSE 383.83359083344834 Test RE 0.998697751277094
62 Train Loss 2.3811495 Test MSE 383.83359083344834 Test RE 0.998697751277094
63 Train Loss 2.3811495 Test MSE 383.83359083344834 Test RE 0.998697751277094
64 Train Loss 2.3811495 Test MSE 383.83359083344834 Test RE 0.998697751277094
65 Train Loss 2.3811495 Test MSE 383.83359083344834 Test RE 0.998697751277094
66 Train Loss 2.3811495 Test MSE 383.83359083344834 Test RE 0.99

50 Train Loss 2.3808422 Test MSE 383.7634923550847 Test RE 0.9986065523943242
51 Train Loss 2.3808422 Test MSE 383.7634923550847 Test RE 0.9986065523943242
52 Train Loss 2.3808422 Test MSE 383.7634923550847 Test RE 0.9986065523943242
53 Train Loss 2.3808422 Test MSE 383.7634923550847 Test RE 0.9986065523943242
54 Train Loss 2.3808422 Test MSE 383.7634923550847 Test RE 0.9986065523943242
55 Train Loss 2.3808422 Test MSE 383.7634923550847 Test RE 0.9986065523943242
56 Train Loss 2.3808422 Test MSE 383.7634923550847 Test RE 0.9986065523943242
57 Train Loss 2.3808422 Test MSE 383.7634923550847 Test RE 0.9986065523943242
58 Train Loss 2.3808422 Test MSE 383.7634923550847 Test RE 0.9986065523943242
59 Train Loss 2.3808422 Test MSE 383.7634923550847 Test RE 0.9986065523943242
60 Train Loss 2.3808422 Test MSE 383.7634923550847 Test RE 0.9986065523943242
61 Train Loss 2.3808422 Test MSE 383.7634923550847 Test RE 0.9986065523943242
62 Train Loss 2.3808422 Test MSE 383.7634923550847 Test RE 0.998

In [11]:
a = 0
for i in range(10):
    a = a + test_re_full[i][-1]
print(a/10)

nan
